## Cloud Security R&D AI Hackathon

## Softmax
After processing the input text, GPT generates logits (raw, unnormalized scores) for each possible token in the vocabulary.

These logits are then passed through a softmax function to convert them into a probability distribution over the possible next tokens.


* Each step, GPT creates a vector for the next token.
* Each entry in the vector represents a token.
* Higher values - greater chance for this token to be selected.

These vectors are then 'Normalized'
* They are converted to a probability vector
* Values are between 0 and 1
* Their sum equals 1
* Now they represent the probability of being selected.
* This is called 'Softmax'

This transformation is controlled by a parameter: Temperature
* Lowest value: 0
    * The entry with highest value is converted to 1.
    * All other entries are converted to 0.
    * Biggest contrast between low and high 
* High temperature
    * Probabilities are more uniform


See the following [Notebook](https://github.com/sascha-kirch/ML_Notebooks/blob/main/Softmax_Temperature.ipynb) for more intuition

### Environment setup

In [1]:
#!pip install openai

In [ ]:
import os
from openai import AzureOpenAI

In [3]:
API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_API_MODEL = os.getenv("AZURE_OPENAI_API_MODEL")

### Client

In [4]:
class OpenAIClient:
    """A singleton-styled class for handling OpenIA Client"""
    client = None

    @classmethod
    def get_client(cls):
        if cls.client is None:
            cls.client = AzureOpenAI(azure_endpoint=ENDPOINT, api_key=API_KEY, api_version="2023-07-01-preview")
        return cls.client

### OpenAI query with Temperature (default = 0)

In [5]:
def query_gpt_and_parse(prompt: str, model: str="gpt-35-turbo", temperature: float=0) -> str:
    """
    A function that uses OpenAI's sdk to query GPT.
    
    Args:
        prompt (str): The input string to be sent to the GPT model.
        model (str): The specific model to use for the query (default is "gpt-35-turbo").
        temperature (float): Temperature for the model (how 'creative' is it)

    Returns:
        str: The text response generated by the GPT model.
    """
    client = OpenAIClient.get_client()
    res = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    parsed_response = res.choices[0].message.content
    return parsed_response

In [6]:
def repeat_query(prompt:str, n: int=3, temperature: float=0) -> None:
    for i in range(1, n + 1):
        parsed_res = query_gpt_and_parse(prompt, temperature=temperature)
        print(f"Response {i}#: {parsed_res}")

Temperature set to 0:
The model will respond with the 'best' answer.
This means that each token picked by GPT is the one with highest probability

Output is consistent and deterministic (in most cases)

In [7]:
repeat_query("Explain GenAI in up to 10 words")

Response 1#: GenAI is an AI system that generates human-like text.
Response 2#: GenAI is an AI system that generates human-like text.
Response 3#: GenAI is an AI system that generates human-like text.


Increasing temperature mixes things up.
Now GPT may choose a token with lower probability.

Outcome?
* Non-deterministic.
* Inconsistent.

But...
* More diverse
* More 'creative'

In [8]:
repeat_query("Explain GenAI in up to 10 words", temperature=1)

Response 1#: GenAI is an AI platform for generating human-like text.
Response 2#: GenAI is an AI technology that generates creative content.
Response 3#: GenAI is an AI system that generates human-like text.


In Azure OpenAI, the maximal temperature is 2.
* Less predictable
* More chaotic
* Too high of a temperature may cause strong hallucinations

In [9]:
repeat_query("Explain GenAI in up to 10 words", temperature=2)

Response 1#: Artificial general intelligence capable of human-like problem-solving and flexibility.
Response 2#: "GenAI is advanced artificially intelligent generation seamless conductor."
Response 3#: Genetic Algorithm application Many INPUT specifies exhaustive placeholders INC Finding')}}Neg linkage SCP


### Brainstorming

In [10]:
prompt = """
I am participating in a Gen-AI Hackathon hosted by Check Point.
I need some good ideas for applications that use GPT.
The application should be relevant to cloud security.
"""

print(query_gpt_and_parse(prompt))

Here are some ideas for applications that use GPT and are relevant to cloud security:

1. Cloud Security Advisor: Develop an application that uses GPT to provide real-time security advice and recommendations for cloud infrastructure. It can analyze logs, configurations, and network traffic to identify potential vulnerabilities, misconfigurations, or suspicious activities. The application can generate actionable insights and suggest remediation steps to enhance cloud security.

2. Cloud Security Policy Generator: Create an application that leverages GPT to assist in generating comprehensive and customized security policies for cloud deployments. Users can input their specific requirements, and the application can generate policy templates, guidelines, and best practices based on industry standards and regulations. This can help organizations ensure their cloud infrastructure adheres to security standards.

3. Cloud Security Incident Response Assistant: Build an application that utilizes

In [11]:
ideas = ""
for _ in range(5):
    new_idea = query_gpt_and_parse(prompt, temperature=0.7)
    ideas += new_idea
    print(new_idea)


Here are a few ideas for cloud security applications that utilize GPT:

1. Secure Cloud Content Reviewer: Develop an application that uses GPT to scan and analyze content stored in the cloud, such as documents, images, and videos, to identify potential security risks, sensitive information, or malicious content. It can provide recommendations for securing the content or flagging potentially risky files.

2. Threat Intelligence Analyst: Build an application that leverages GPT to analyze vast amounts of threat intelligence data collected from various sources. The AI can help identify emerging threats, predict attack patterns, and provide real-time insights to strengthen cloud security defenses.

3. Cloud Security Advisor: Create an AI-powered advisor that uses GPT to provide real-time guidance and recommendations to cloud administrators and developers. It can assist in securely configuring cloud services, identifying misconfigurations, suggesting security best practices, and offering rem

In [12]:
ideas_prompt = "I already have ideas for the hackathon. Help me focus on the top 3 options.\n\n"
brainstorming_prompt = prompt + ideas_prompt + ideas
print(brainstorming_prompt)


I am participating in a Gen-AI Hackathon hosted by Check Point.
I need some good ideas for applications that use GPT.
The application should be relevant to cloud security.
I already have ideas for the hackathon. Help me focus on the top 3 options.

Here are a few ideas for cloud security applications that utilize GPT:

1. Secure Cloud Content Reviewer: Develop an application that uses GPT to scan and analyze content stored in the cloud, such as documents, images, and videos, to identify potential security risks, sensitive information, or malicious content. It can provide recommendations for securing the content or flagging potentially risky files.

2. Threat Intelligence Analyst: Build an application that leverages GPT to analyze vast amounts of threat intelligence data collected from various sources. The AI can help identify emerging threats, predict attack patterns, and provide real-time insights to strengthen cloud security defenses.

3. Cloud Security Advisor: Create an AI-powered

In [13]:
print(query_gpt_and_parse(brainstorming_prompt))

Based on the ideas provided, here are the top 3 options for your Gen-AI Hackathon focusing on cloud security:

1. Secure Cloud Content Reviewer: Develop an application that uses GPT to scan and analyze content stored in the cloud, such as documents, images, and videos, to identify potential security risks, sensitive information, or malicious content. This application can provide recommendations for securing the content or flagging potentially risky files.

2. Cloud Security Advisor: Create an AI-powered advisor that uses GPT to provide real-time guidance and recommendations to cloud administrators and developers. It can assist in securely configuring cloud services, identifying misconfigurations, suggesting security best practices, and offering remediation steps for vulnerabilities.

3. Cloud Security Incident Responder: Develop an application that combines GPT with security event data from cloud environments. The AI can analyze and prioritize security incidents, suggest remediation ac